# Fase 3: Evaluación de Diferencias en Reservas de Vuelos por Nivel Educativo

Objetivo del Ejercicio: Utilizando un conjunto de datos que hemos compartido, se busca evaluar si existen diferencias significativas en el número de vuelos reservados según el nivel educativo de los clientes. Para ello, los pasos que deberas seguir son:

1. Preparación de Datos: 

    - Filtra el conjunto de datos para incluir únicamente las columnas relevantes: 'Flights Booked' y 'Education'.

2. Análisis Descriptivo: 

    - Agrupa los datos por nivel educativo y calcula estadísticas descriptivas básicas (como el promedio, la desviación estandar, los percentiles) del número de vuelos reservados para cada grupo.

3. Prueba Estadística:

    - Realiza una prueba de A/B testing para determinar si existe una diferencia significativa en el número de vuelos reservados entre los diferentes niveles educativos.

### Importamos las librerías que necesitamos

In [2]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
from src import support as sp

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# # Evaluar linealidad de las relaciones entre las variables
# # y la distribución de las variables
# # ------------------------------------------------------------------------------
# import scipy.stats as stats
# from scipy.stats import chi2_contingency, ttest_ind
# from scipy.stats import shapiro, poisson, chisquare, expon, kstest

In [3]:
# Cargamos los datos generados en la Fase 1
df_combined = pd.read_csv('../data/df_combined.csv', index_col=0)

In [4]:
# Mostramos las primeras filas del DataFrame
df_combined.head(3) 

,loyalty_number,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,points_redeemed,dollar_cost_points_redeemed,province,city,postal_code,gender,education,salary,marital_status,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month,cancellation_year,cancellation_month
0,100018,2017,1,3,0,3,1521,152.0,0,0,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8,NaN,NaN
1,100102,2017,1,10,4,14,2030,203.0,0,0,Ontario,Toronto,M1R 4K3,Male,College,NaN,Single,Nova,2887.74,Standard,2013,3,NaN,NaN
2,100140,2017,1,6,0,6,1200,120.0,0,0,British Columbia,Dawson Creek,U5I 4F1,Female,College,NaN,Divorced,Nova,2838.07,Standard,2016,7,NaN,NaN


- Filtra el conjunto de datos para incluir únicamente las columnas relevantes: 'Flights Booked' y 'Education'.

* Si solo filtramos por esas 2 columnas, luego perdemos la referencia de quien son estos clientes, por lo cuál, filtraremos por 3 columnas

In [5]:
# Filtrar el conjunto de datos para incluir las columnas 'loyalty_number', 'flights_booked' y 'education'
df_filtered = df_combined[['loyalty_number', 'flights_booked', 'education']]
df_filtered.shape

(401688, 3)

- Agrupa los datos por nivel educativo y calcula estadísticas descriptivas básicas (como el promedio, la desviación estandar, los percentiles) del número de vuelos reservados para cada grupo.

In [6]:
# Agrupamos por loyalty_number y education, que son los valores que se repiten, para extraer las estadísticas descriptivas de flights_booked por cliente.
df_filtered.groupby(['loyalty_number', 'education'])['flights_booked'].describe().sample(10)

,,count,mean,std,min,25%,50%,75%,max
loyalty_number,education,,,,,,,,
230429,Bachelor,24.0,6.958333,5.937934,0.0,1.75,6.5,10.25,21.0
779882,Doctor,24.0,5.416667,5.149223,0.0,0.75,4.5,9.25,16.0
239955,Bachelor,24.0,7.166667,6.363392,0.0,0.00,7.0,13.00,19.0
718655,Bachelor,24.0,1.666667,3.102126,0.0,0.00,0.0,1.00,9.0
138993,Bachelor,24.0,5.083333,6.870964,0.0,0.00,0.0,9.00,21.0
703109,Bachelor,24.0,7.500000,5.047600,0.0,3.00,7.5,11.25,17.0
684305,College,24.0,4.583333,4.353077,0.0,0.00,4.5,8.00,12.0
128975,Bachelor,24.0,5.625000,5.792142,0.0,0.00,4.5,9.50,19.0
687401,College,24.0,6.166667,4.904893,0.0,2.75,6.0,9.25,16.0


- Realiza una prueba de A/B testing para determinar si existe una diferencia significativa en el número de vuelos reservados entre los diferentes niveles educativos.

In [7]:
# Primero filtramos el conjunto de datos 
df_filtered.groupby(['loyalty_number', 'education'])['flights_booked'].sum().reset_index().sample(10)

,loyalty_number,education,flights_booked
8474,556188,Bachelor,152
6223,435249,Bachelor,133
1642,189197,Bachelor,164
5096,375105,Bachelor,120
11247,703852,Bachelor,45
2161,215952,Master,93
3890,310833,Bachelor,0
9352,603079,Doctor,128
16339,976834,Bachelor,11
5518,398924,Bachelor,146


In [8]:
# Guardamos este DataFrame en una variable para usarlo en nuestro A/B testing
df_filtered_ab = df_filtered.groupby(['loyalty_number', 'education'])['flights_booked'].sum().reset_index()
df_filtered_ab.tail(3)

,loyalty_number,education,flights_booked
16734,999940,Bachelor,63
16735,999982,College,22
16736,999986,Bachelor,121


In [9]:
df_filtered_ab.education.value_counts() # Miramos como están distribuidos los niveles de educación

education
Bachelor                10475
College                  4238
High School or Below      782
Doctor                    734
Master                    508
Name: count, dtype: int64

El orden de los niveles de educación de menor a mayor es:

1. **High School or Below** (Secundaria o menos)
2. **College** (Universidad, generalmente implica que no se completó un título universitario)
3. **Bachelor** (Licenciatura)
4. **Master** (Máster)
5. **Doctor** (Doctorado)

### Ahora vamos a crear nuestro test_group, categorizando la columna education en 2 grupos: 
    * GRUPO A: Clientes con nivel de educación inferior: High School or Below y College
    * GRUPO B: Clientes con nivel de educación superior: Bachelor, Doctor y Master

In [10]:
# Usamos el apply para aplicar nuestra función que categoriza las distintos niveles de education
group_a = ['High School or Below', 'College']
group_b = ['Bachelor', 'Master', 'Doctor']

df_filtered_ab['test_group'] = df_filtered_ab['education'].apply(lambda x: sp.categorize(x, group_a, group_b))

In [12]:
df_filtered_ab.sample(5)

,loyalty_number,education,flights_booked,test_group
13615,831035,College,172,group_a
16490,985839,College,111,group_a
612,133128,Bachelor,122,group_b
3064,265297,Bachelor,38,group_b
10679,673424,Master,143,group_b


Lo primero que debemos hacer en un A/B testing es verificar que los datos son coherentes y están listos. En este caso, ya tenemos este paso concluído.

### Planteamos las hipótesis

- Hipótesis nula (H0): No existe una diferencia significativa en el número de vuelos reservados entre los diferentes niveles educativos de los grupos A y B.
- Hipótesis alternativa (H1): Existe una diferencia significativa en el número de vuelos reservados entre los diferentes niveles educativos de los grupos A y B.

### Calculamos la tasa de conversión 

- Para el grupo "ad", la tasa de conversión es el número de usuarios convertidos dividido por el número total de usuarios en ese grupo.
- Para el grupo "psa", realiza el mismo cálculo.